In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp profile

# Profile

> Set of functions for calculating roughness parameters along profiles 

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from rough.data import *

In [ ]:
#| export
def Ra(im, #Numpy array or arraylike
       axis = 1, #Default to Ra of rows
       norm = True #Normalize the profile by subtracting the mean 
      ):
    '''
    Calculates Mean Absolute Roughness (Ra) along given axis. Defined as the average deviation of absolute height values from the mean line.
    '''
    if norm:
        im = im - np.mean(im, axis = axis, keepdims = True)
    return np.mean(np.absolute(im), axis = axis)

In [ ]:
#| export
def Rms(im, #Numpy array or array like
        axis = 1, #Default to Rms of rows
        norm = True #Normalize the profile by subtracting the mean
       ):
    '''
    Calculates Root Mean Square Roughness (Rms) along given axis. Defined as the root mean square of deviations of height from the mean line of a given profile. 
    '''
    if norm:
        im = im - np.mean(im, axis = axis, keepdims = True)
    return np.sqrt(np.mean(np.square(im), axis = axis))

# Rotational parameters
>
>The following functions return roughness parameters as rotated around the central point of the array.
>Useful for investigating how roughness changes with respect to direction.
>Uses OpenCV/imutils to rotate arrays,because the initial image/array dimensions are preserved 
>by default it gives an easy way to use the diameter of the central circle as the profile. 
>As these are mostly thin wrappers around standard parameters, it is recommended to generate
>an array of rotational profiles using `gen_rot_prof` first and then using standard
>paramaters above. This is highly recommended for very large arrays.

In [ ]:
def rot_Ra(array, axis=1, deg = 180, increment=1):
    '''
    Calculated rotational Ra for a given array
    '''
    return Ra(gen_rot_prof(array,deg=deg,increment=increment), axis = axis)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()